# Stochastic modeling
## Gillespie Algorithm

In [ ]:
using StatsBase # Weights() and sample()
using Random    # randexp()
using Plots

In [ ]:
#=
Stochastic chemical reaction: Gillespie Algorithm (direct method)
Adapted from: Chemical and Biomedical Enginnering Calculations Using Python Ch.4-3
=#
function ssa_direct(model, u0, tend, p, stoich; tstart=zero(tend))
    t = tstart   # Current time
    ts = [t]     # Time points
    u = copy(u0) # Current state
    us = copy(u) # Record of states
    while t < tend
        a = model(u, p, t)               # propensities
        dt = randexp() / sum(a)          # Time step
        du = sample(stoich, Weights(a))  # Choose the stoichiometry for the next reaction
        u .+= du  # Update state
        t += dt   # Update time
        
        us = [us u]  # Append state variable to record
        push!(ts, t) # Append time point to record
    end
    # Trasnpose to make column as variables, rows as observations
    us = collect(us')
    return (t = ts, u = us)
end

In [ ]:
#=
Stochastic chemical reaction: Gillespie Algorithm (first reaction method)
Adapted from: Chemical and Biomedical Enginnering Calculations Using Python Ch.4-3
=#
function ssa_first(model, u0, tend, p, stoich; tstart=zero(tend))
    t = tstart   # Current time
    ts = [t]     # Time points
    u = copy(u0) # Current state
    us = copy(u) # Record of states
    while t < tend
        a = model(u, p, t)  # propensities of reactions
        # dts from all reactions
        dts = randexp(length(a)) ./ a
        # Choose the reaction 
        i = argmin(dts)
        dt = dts[i]
        du = stoich[i]
        # Update state and time
        u .+= du
        t += dt
        us = [us u]  # Append state variable to record
        push!(ts, t) # Append time point to record
    end
    # Make column as variables, rows as observations
    us = collect(us')
    return (t = ts, u = us)
end

In [ ]:
#=
Reaction of A <-> B with rate constants k1 & k2
=#
"Propensity model for this reaction"
model(u, p, t) = [p.k1 * u[1],  p.k2 * u[2]]

In [ ]:
parameters = (k1=1.0, k2=0.5, stoich=[[-1, 1], [1, -1]])
u0 = [200, 0]
tend = 10.0

soldirect = ssa_direct(model, u0, tend, parameters, parameters.stoich)
solfirst = ssa_first(model, u0, tend, parameters, parameters.stoich)

In [ ]:
plot(soldirect.t, soldirect.u, xlabel="time", ylabel="# of molecules", title = "SSA (direct method)", label=["A" "B"])

In [ ]:
plot(solfirst.t, solfirst.u, xlabel="time", ylabel="# of molecules", title = "SSA (1st reaction method)", label=["A" "B"])

In [ ]:
# Running an ensemble of simulations
numRuns = 50
sol = ssa_direct(model, u0, tend, parameters, parameters.stoich)
p = plot(sol.t, sol.u, linecolor=[:blue :red], label=["A" "B"], linealpha=0.2,
    xlabel="time", ylabel="# of molecules", 
    title = "SSA (1st reaction method) ensemble")
for i in 1:numRuns-1
    sol = ssa_direct(model, u0, tend, parameters, parameters.stoich)
    plot!(p, sol.t, sol.u, linecolor=[:blue :red], linealpha=0.2, label=false)
end

p